### C code for comparison with Numpy/Cython

This code is used in here: [python is slow][slowpy]


[slowpy]: ./notes.ipynb#python_is_slow

In [46]:
//%cflags:-shared -O3 -ffast-math

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

#define MAX_VALUE 100
#define STEP_MUL_RATIO 2
#define MAX_TEST_SIZE pow(2, 22)
#define INITIAL_TEST_SIZE pow(2, 6)


void add_one(long* test_arr, unsigned long s){
    unsigned long i;
    for (i = 0; i < s; i++) {
        test_arr[i] = test_arr[i] + 1;
    }
}

unsigned long next_test_size(unsigned long pre_size){
    return pre_size*STEP_MUL_RATIO;
}

void print_arr(long *arr, unsigned long arr_size){
    printf("\n\t");  
    int i;
    for (i = 0; i < arr_size; i++) {
        printf("%li ", arr[i]);
    }
    printf("\n");
}

void run_test(unsigned long test_size){
    long *test_arr = (long *)malloc(sizeof(long)*test_size);
    unsigned long i;
    for (i = 0; i < test_size; i++) {
        test_arr[i] = rand()%MAX_VALUE;
    }
    
    /* printf("\nBEFORE:");
    print_arr(test_arr, test_size); */
    
    clock_t t0 = clock();
    add_one(test_arr, test_size);
    clock_t t1 = clock();

    /* printf("\nAFTER:");
    print_arr(test_arr, test_size); */

    double exec_time_s = (double)(t1 - t0) / CLOCKS_PER_SEC;
    double exec_time_ms = exec_time_s*1e3;
    printf("(%lu, %.3f), ", test_size, exec_time_ms);
}

int main(void) {
    srand(0);
    unsigned long s;
    unsigned int i = 0;
    printf("c_results = { \"C implementation\": [");
    for (s = INITIAL_TEST_SIZE; s < MAX_TEST_SIZE; s = next_test_size(s)){
        if (i%4 == 0)
            printf("\n\t");
        run_test(s);
        i++;
    }
    printf("\n]}");
    return 0;
}

c_results = { "C implementation": [
	(64, 0.003), (128, 0.000), (256, 0.001), (512, 0.000), 
	(1024, 0.001), (2048, 0.001), (4096, 0.003), (8192, 0.003), 
	(16384, 0.010), (32768, 0.013), (65536, 0.033), (131072, 0.082), 
	(262144, 0.180), (524288, 0.427), (1048576, 0.865), (2097152, 1.573), 
]}

In [167]:
//%cflags: -O3

#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>

#define N 3
#define M 1000

float** init_array(int m, int n){
    float* cells = calloc(n*m, sizeof(float));
    float** array = malloc(m*sizeof(float*));
    int i;
    for (i=0; i<m; ++i){
        array[i] = cells + i*n;
    }
    return array;
}

void random_fill(float** array, int m, int n){
    int i, j;
    for (i=0; i < m; i++){
        for (j = 0; j < n; j++){
            array[i][j] = rand()/(float)RAND_MAX;
        }
    }
}

void print_arr(float** arr, int m, int n){
    printf("\n");  
    int i, j;
    for (i = 0; i < m; i++) {
        printf("\t[");
        for (j = 0; j < n; j++){
            printf("%f, ", arr[i][j]);
        }
        printf("]\n");
    }
    printf("\n");
}


float** pairwise(float** test_arr){
    float** res_array = init_array(M, M);
    int i, j, k;
    float d, tmp;
    for (i = 0; i < M; i++){
        for (j=0; j < M; j++){
            d = 0.0;
            for (k = 0; k < N; k++){
                tmp = test_arr[i][k] - test_arr[j][k];
                d += tmp * tmp;
            }
            res_array[i][j] = sqrt(d);
        }
    }
    return res_array;
}


void run_test(){
    float** test_arr = init_array(M, N);
    random_fill(test_arr, M, N);
    
    //printf("\nBEFORE:");
    //print_arr(test_arr, M, N);
    
    clock_t t0 = clock();
    float** r = pairwise(test_arr);
    clock_t t1 = clock();
    
    //printf("\nAFTER:");
    //print_arr(r, M, M);

    double exec_time_s = (double)(t1 - t0) / CLOCKS_PER_SEC;
    double exec_time_ms = exec_time_s*1e3;
    printf("Exec time = %.3f ms", exec_time_ms);
}

int main(void) {
    srand(0);
    run_test();
    return 0;
}

Exec time = 4.313 ms